In [16]:
import sys
sys.path.append('..')
import processing
import configs.common as cc
import train
from train import new_model
import torch
import configs.paths as paths
import argparse
import os
from collections import Counter
from generate import generate

data_root = "/home/s203861/midi-classical-music/np_data/data"
model_name = "xlstm"

if model_name == "mamba":
    model = new_model("mamba")
    model.load_state_dict(torch.load(cc.config.models.mamba))
    model.to('cuda')
if model_name == "xlstm":
    model = new_model("xlstm")
    model.load_state_dict(torch.load(cc.config.models.xlstm))
    model.to('cuda')
if model_name == "transformer":
    model = new_model("transformer")
    model.load_state_dict(torch.load(cc.config.models.transformer))
    model.to('cuda')

In [17]:
import time

loader = processing.DatasetLoader(data_root)
dataloader = loader.get_dataloader_full()

num_outputs = 100
inference_times = []
model.train()

for src, trg, meta in dataloader:
    start_time = time.time()
    
    for _ in range(num_outputs):
        with torch.no_grad():
            model(src, meta)
    
    end_time = time.time()
    
    total_time = end_time - start_time
    avg_time_per_output = total_time / num_outputs
    
    inference_times.append(avg_time_per_output)
    
    break

# Print the average inference time for 100 outputs
print(f"Average inference time per output: {avg_time_per_output:.6f} seconds: total time {total_time}")

Average inference time per output: 0.072655 seconds: total time 7.265500783920288


In [9]:
torch.cuda.memory_allocated()

835563520